In [1]:
import pandas as pd
import requests
from requests import request
from bs4 import BeautifulSoup
from multiprocessing.dummy import Pool as ThreadPool
import functools
import time
from tqdm.notebook import tqdm
import sys
from itertools import chain
import re
import functools
import pymorphy2

In [2]:
import pandas as pd 
import datetime
import numpy as np 
import matplotlib.pyplot as plt 

In [3]:
import pandas as pd
from bs4 import BeautifulSoup
import bs4
from multiprocessing import Pool, Lock, Value
from time import sleep
from xgboost import XGBClassifier
from sklearn.model_selection import cross_validate
import base64
import numpy as np
import zlib
from itertools import product
import re

In [4]:
import pymorphy2  # $ pip install pymorphy2
  
def pos(word, morth=pymorphy2.MorphAnalyzer()):
    "Return a likely part of speech for the *word*."""
    return morth.parse(word)[0].tag.POS
  
words = "Однако я так и не смог закончить".split()
functors_pos = {'INTJ', 'PRCL', 'CONJ', 'PREP'}  # function words
# p2 = [word for word in p1 if (pos(word) not in functors_pos and len(word)>1)]
# -> я смог закончить

In [5]:
list_mest = ['я', 'ты', 'он', 'она', 'оно', 'мы',
             'вы', 'они','мой', 'твой', 'свой', 'ваш', 'наш', 'его', 'её', 'их',
            'все','быть','может', 'можно', 'очень','нужно','будет','уже','могут',
            'кто', 'что', 'какой', 'чей', 'где', 'который', 'откуда', 'сколько', 'каковой', 'каков', 'зачем',
            'меня','мне','того', 'этого', 'этот','ее','вы','более', 'было', 'вам', 'все', 'вы',  
             'которые', 'нет',  'том', 'этом',
            'вас', 'вообще', 'всего', 'вы',
            'себе', 'себя', 'ему', 'тебе', 'тебя','был']

In [6]:
morph = pymorphy2.MorphAnalyzer()

In [7]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler

In [8]:


def batch_generator(X, y, shuffle=True, batch_size=1):
    """
    Гератор новых батчей для обучения
    X          - матрица объекты-признаки
    y_batch    - вектор ответов
    shuffle    - нужно ли случайно перемешивать выборку
    batch_size - размер батча ( 1 это SGD, > 1 mini-batch GD)
    Генерирует подвыборку для итерации спуска (X_batch, y_batch)
    """
    ind = np.array(range(y.shape[0]), dtype = int)
    if(shuffle):
        ind = np.random.permutation(y.shape[0])
    for i in range(len(y)//batch_size):
        X_batch = X[ind[i*batch_size:(i+1)*batch_size],:]
        y_batch = y[ind[i*batch_size:(i+1)*batch_size]]
        yield (X_batch, y_batch)

In [9]:
from sklearn.base import BaseEstimator, ClassifierMixin


def sigmoid(x):
    """
    Вычисляем значение сигмоида.
    X - выход линейной модели
    """
    # Your code Here
    sigm_value_x = 1/(1 + np.exp(-(x)))
    return sigm_value_x


class MySGDClassifier(BaseEstimator, ClassifierMixin):
    def __init__(self, batch_generator, C=1, alpha=0.01,
                 max_epoch=10, model_type='lin_reg', batch_size=1, th=0.5):
        """
        batch_generator -- функция генератор, которой будем создавать батчи
        C - коэф. регуляризации
        alpha - скорость спуска
        max_epoch - максимальное количество эпох
        model_type - тим модели, lin_reg или log_reg
        """
        self.th = th
        self.C = C
        self.alpha = alpha
        self.max_epoch = max_epoch
        self.batch_generator = batch_generator
        self.errors_log = {'iter' : [], 'loss' : []}  
        self.model_type = model_type
        self.batch_size = batch_size

    def calc_loss(self, X_batch, y_batch):
        """
        Считаем функцию потерь по батчу 
        X_batch - матрица объекты-признаки по батчу
        y_batch - вектор ответов по батчу
        Не забудте тип модели (линейная или логистическая регрессия)!
        """
        if (self.model_type == 'lin_reg'):
            loss = 1 / y_batch.shape[0] * ((np.dot(X_batch, self.weights.reshape(-1, 1)) - y_batch) ** 2).sum() 
            + 1 / self.C * ((self.weights * self.weights).sum())
            return loss
        else:
            sigm = np.arange(y_batch.shape[0], dtype = float)
            for i in range(y_batch.shape[0]):
                sigm[i] = sigmoid((X_batch[i, : ] * self.weights).sum())
            loss = - (y_batch * np.log(sigm) + (1 - y_batch) * np.log(1 - sigm)).sum() / y_batch.shape[0] 
            + 1 / self.C * ((self.weights * self.weights).sum())
            return loss

    def calc_loss_grad(self, X_batch, y_batch):
        """
        Считаем  градиент функции потерь по батчу (то что Вы вывели в задании 1)
        X_batch - матрица объекты-признаки по батчу
        y_batch - вектор ответов по батчу
        Не забудте тип модели (линейная или логистическая регрессия)!
        """
        if (self.model_type == 'lin_reg'):
            loss_grad = (2/y_batch.shape[0])*np.dot((np.dot(X_batch, self.weights.reshape(-1,1)) 
                                        - y_batch.reshape(-1,1)).T,X_batch).sum(axis = 0) + 2/self.C*self.weights
            return loss_grad.reshape(-1)
        else:
            sigm = np.arange(y_batch.shape[0], dtype = float)
            for i in range(y_batch.shape[0]):
                sigm[i] = sigmoid((X_batch[i, : ]*self.weights).sum())
            loss_grad = np.dot(X_batch.T,(sigm - y_batch).reshape(-1,1)).reshape(self.weights.shape[0])
            + 2/self.C*self.weights
            return loss_grad

    def update_weights(self, new_grad):
        """
        Обновляем вектор весов
        new_grad - градиент по батчу
        """
        self.weights -= self.alpha *new_grad

    def fit(self, X, y):
        '''
        Обучение модели
        X - матрица объекты-признаки
        y - вектор ответов
        '''
        # Нужно инициализровать случайно веса
        X = np.hstack((np.ones((X.shape[0], 1)), X))
        self.weights = np.random.random(X.shape[1])
        for n in range(0, self.max_epoch):
            new_epoch_generator = self.batch_generator(X, y, batch_size = self.batch_size)
            for batch_num, new_batch in enumerate(new_epoch_generator):
                X_batch = new_batch[0]
                y_batch = new_batch[1]
                batch_loss = self.calc_loss(X_batch, y_batch)
                batch_grad = self.calc_loss_grad(X_batch, y_batch)
                self.update_weights(batch_grad)
                # Подумайте в каком месте стоит посчитать ошибку для отладки модели
                # До градиентного шага или после
                self.errors_log['iter'].append(batch_num)
                self.errors_log['loss'].append(batch_loss)
        return self

    def predict(self, X):
        '''
        Предсказание класса
        X - матрица объекты-признаки
        Не забудте тип модели (линейная или логистическая регрессия)!
        '''
        X = np.hstack((np.ones((X.shape[0], 1)), X))
        if (self.model_type == 'lin_reg'):
            y_hat = np.dot(X, self.weights)/np.sum(X)
        else:
            sigm = [[] for i in range(X.shape[0])]
            for i in range(X.shape[0]):
                sigm[i] = sigmoid((X[i, : ]*self.weights).sum())
            y_hat = (np.array(sigm))
        # Желательно здесь использовать матричные операции между X и весами, например, numpy.dot 
        return y_hat

In [11]:
import re
from pymystem3 import Mystem
doc_to_t = {}
m = Mystem()
with open('docs_titles.tsv') as f:
    for num_line, line in enumerate(f):
        if num_line == 0:
            continue
        if num_line == 0:
            continue
        data = line.strip().split('\t', 1)
        
        doc_id = int(data[0])
        if len(data) == 1:
            title = ''
        else:
            title = data[1]
#         cur = re.split(r' ',title)
        r = [ i.lower() for i in re.split(r'\W+',title) if i]
        p = [re.sub(r'[^А-я,0-9]', '', i) for i in r]
        p1 = list(filter(lambda i: i, p))
        cur = [word for word in p1 if (pos(word) not in functors_pos and len(word)>1)]
        cur = [u.lower() for u in cur if not u.lower() in list_mest]
        cur = m.lemmatize(' '.join(cur))
#         cur = [morph.parse(o)[0].normal_form for o in cur]
        
        title = ''
        for i in cur:
            if len(i)>2:
                title += i + ' '
        print(title)
        doc_to_t[doc_id] = title
print (len(doc_to_t))

ваз 21213 замена подшипник ступица нива 
ваз 2107 опт сочи сравнивать цена купить потребительский товар 
купить ступица лад калина2 трансмиссия переходный ступица цена замена тюнинг 
классика 21010 21074 
ступица нива замена подшипник свой рука 
ваз 2110 
обзор подшипник полуось ваз 2101 2121 2123 
купить подшипник ступица страница 
автомобильный интернет портал отзыв владелец ваз 2121 нива 2007 год 
новость сообщение официальный группа вконтакте торговый компания магазин тольятти 
инструкция замена подшипник передняя ступица ивеко дейли 
ступица страница 
маааленькая проблемка бортжурнал авток 2160 1994 год 
разгружать полуось нива шлиц 765 
прошивка нива скачать файлообменник 
страница раздел каталог подвеска 
продавать нива 2121 
рекомендация проведение независимый экспертиза осаго 
втулка подшипник омск сравнивать цена поставщик промышленный товар 
шевроле блейзер елабуга руководство эксплуатация инструкция совет 
привод шрус пыльник шрусый страница 
автомобиль маз калининград инте

гастрит зуб 
болезнь орган дыхание продолжение 
паразитарный теория рак просмотр тема синий йод 
материнство грипп орви 
лечить простуда 
народный рецепт кашель 
опасный кашель беременность причина появление способ лечение 
горчичник ларингит согревать компресс горло 
метод лечение ларингит народный средство домашний условие 
лечебный сила мед 
детский ларингит медицина педиатрия 
лечение чеснок чеснок целебный приправа 
аденовирусный инфекция 
протодиакон андрей кураев второе пришествие апокриф храм смоленский икона божий мать 
канделак татья 
анализ ттг сдавать правильно сдавать день цикл 
родиола розовый женьшень красиво здоровый образ жизнь 
реконструкция мостовый перегружатель грузоподъемность тонна грузовой атп 

всн 110 технический указание проектирование строительство фундамент опора мост сборная железобетонный оболочка скачать бесплатно 
приготавливать кабачок гастрит 
писать стих 
франция эпоха ренессанс 
первый алтайский грамматика алтай туристский туристический портал 
разр

финансовый грамотность вариант создание инвестиционный портфель беседка мозырьавто мозырский автомобильный форум мозырь авто 
инвестиция зарубежный недвижимость финансы деньги рынок 
управление реальный инвестиция управлять они эффективно 
связь ставка цена нефть 
внебюджетный фонд профсоюз социальный политика 
агропром 
четырнадцатый инвестиционный отчт 
сегодня выгодно безопасно вкладывать деньги что начинать 
куда выгодно вкладывать сбережение наступать год 
роль иностранный инвестиция экономический развитие казахстан 
вкладывать деньги совет инвестор 
искать советник практика использовать собственный совет 
два способ инвестирование 
оценка реализация инвестиционный проект сфера жилищный коммунальный хозяйство пример деятельность предприятие жкх красноселькупский район ямало ненецкий автономный округ 
инвестирование выгодно вкладывать пиф 
выгодный инвестиция недвижимость африка бизнес африка 
новость 
стоить вкладывать деньги пиф миф реальность 
инвестиция сбережение курсовой рабо

выдить тула чаво полный бессмертие саратов комплект 3трейлер обзор разбогатеть биржа ошибка скачивание музыка почему притормаживать список изменение 
скачать игра гта сан андреас торрент скачать игра гта сан андреас торрент скачать гта сан андреас торрент минимальный версия игра скачать скачать устанавливать игра полный версия скачать оригинальный версия игра скачать игра английский версия скачать полный версия торрент скачать полный версия легко быстро скачать торрент полный русский версия скачать торрент бета версия чит код гта сан андреас ходить деревенщина чит код гта сан андреас взять гонка код гта сан андреас ходить голай код гта сан андреас человек ходить 
получать 100 прохождение общий каталог статья 
выбирать тюль дома 
длина штора пол сантиметр оставлять фото 
дизайн штора кухня кухонный занавеска тюль изделие люверс другой проект видео фото 
супер крем рост грудь купить обязательно дневник пользователь 
длина форма штора полезно знать штора компания дизайн пошив штора штора 

аллергический насморк лечение народный средство медикамент лечение насморк гайморит 
сухость нос причина возникновение метод лечение 
срочно насморк месяц 
избавляться аллергия домашний условие 
лечение ринит грудничок 
лекарственный аллергия совет специалист аллергология 
аллергический дерматит грудничок вид кожный заболевание 
побеждать ринит острый форма аллергия 
искривление носовой перегородка хронический ринит медицина предмет реферат учебный материал студент школьник 100пудов 
хорошо завтра собираться столица смочь посмотреть 
острый аллергический состояние часть российский 
вазомоторный ринит симптом лечение ребенок 
насморк время беременность 
охрипнуть голос лечить 
лающий кашель ребенок причина лечение 
карта сайт 
острый респираторный заболевание 
болезнь ухо горло нос 
насморк беременность триместр лечить беременность роды 
гость екатеринбург онлайн 
сопли ребнка месяц лечить современный средство борьба насморк 
стена вконтакте 
хорошо вызывать врач дом 
способ диагностика

...

самп сервер сдавать правый самп сервер сервер самп исчезать машина исправлять одинаковый машина везде сохранять тюнинговый машина припарковывать машина отход застраховывать машина списой машина машина милый машина паркур машина затраховать машина тюнинговать машина выглядеть машина бмв 
мэтр мировой психология редакция проф менделевич руководство аддиктология 
хардкорный мод 
щлково просмотр тема мой проза 
лекция щедровицкий организация руководство управление хрестоматия организационный менеджмент теория бизнес инжиниринг группа 
мозаика стекло кальян свой рука статья обсуждение российский сервис онлайн дневник 
понимать менять шрус 
чудак гифок ообщество вконтакте статистика 
купить купон глубокий химчистка полировка кузов покрытие жидкий стекло компания сервис скидка город москва 612 руб экономия 588 руб 
физра 
необычный газ нижний новгород 
уход кожа лицо метод слоеный пирог 
полироль жидкий стекло бортжурнал 2015 год 
живой планета проза история 
захарченко поезд уходить 
техноло

отпетый напарник 2016 кинокрад смотреть онлайн качество 720 
джеки чан фильм джеки чан фильмография смотреть хороший фильм онлайн хороший качество бесплатно регистрация смотреть онлайн бесплатно 
джеки чан страница боевой искусство железный фактор 
джеки чан один праздник двое 
приключение джеки чан весь сезон смотреть онлайн 
джеки чан состариться успевать повзрослеть яплакать 
кино доспехи бог смотреть онлайн 720 русский язык кинодрайв 
змея тень орел джеки чан 
луи армстронг хит роуд джек бесплатно скачать песня 320 
джеки чан сыграть англоязычный комедия 
джеки чан дневник пользователь дневник женский социальный сеть 
джеки чан пропадать семья 2003 онлайн кинотеатр живой кино фильм онлайн бесплатно 
выходить трейлер фильм меч дракон джеки чан шивон джон кьюсак 
джеки чан удостаиваться два сертификат книга рекорд гиннес интересный факт познавательный статья цифра новость 
джеки чан ребенок юмор болталка флудилка игровой 
вечерний ургант джеки чан 
джеки чан открывать азербайджан 
пе

удалять вирус троян всплывать окно попапа шпионский программа прочий гадость компьютер скачать хороший бесплатный антивирус 
опыт блокировка рекламный окно помощь 
программа блокировка всплывать окно любой другой вид реклама 
реклама браузер убирать всплывать окно реклама браузер ремонт компьютер троещин дом компьютерный помощь диагностика компьютер троещина 
блокировать реклама 
убирать реклама браузер навсегда удалять всплывать окно весь сайт 
почистить браузер вирус реклама программа удаление реклама браузер компьютер 
вирусный реклама браузер избавляться помощник интернет 
блокирование всплывать окно реклама 
просмотр тема защищать компьютер рекламный баннер 
скачать бесплатный программа блокировка реклама интернет 
помогать избавляться всплывать реклама браузер заявка 146957 
защита реклама всплывать окно 
ответ подсказывать какой нибудь хороший бесплатный программа защита интернет реклама всплывать окно 
браузер выскакивать реклама избавляться она 
хороший средство удаление вредо

форум автомобильный диагност диагностика ремонт автомобиль авторский статья архив статья 2009 предыдущий 
менять цепь грм змз 406 страница раздел змз 406 клуб газелист 
форум санта автоклуб греть печка 
таблица совместимость диск литой стальной диск колесо советск шина литой диск тест полезный совет 
тюнинг омск сравнивать цена купить потребительский товар 
блог сообщество 102 
тест драйв гибридный 350 
240 210 
ремонт двигатель цена заказывать полтава 4328217 
аксессуар 
сравнительный расход топливо автомобиль тюнинг ссср 
номер двигатель мерседес 
чемпионат мир 2008 архив страница форум 
общий 
включаться вентилятор радиатор клуб любитель ситроен 
бензин архив форум ниссан альмера классик 
видео книга пособие автомаляр инструкция совет мануал 
двигатель тойота вист форум 
установка контроллер давление масло 313 611 
запчасть казахстан сравнивать цена купить потребительский товар 
автовопрос архив страница рыбалка образ жизнь 
класс ошибка р1617 форум автодиагностика автосканер ремонт

творог полезный свойство калорийность применение приготовление польза вред продукт творог молочный продукт леди 
творог польза вред выбирать продукт фитнес здоровье красота диета 
делать кальцинированный творог кто полезный 
полезный свойство творог 
ответ полезный сыворотка творог пить 
ответ полезный творог 
трава стевий заменитель сахар польза вред применение блог ирина зайцева 
творог польза вред организм мужчина женщина полезный свойство диетический творог показание противопоказание ребенок 
творог завтрак полезный похудение 
выбирать вкусный полезный творог продукт питание 
сюжет официальный сайт программа здоровье жить елена малышева 
полезный творог полезный творог завтрак здоровый питание 
творог польза вред выбирать продукт фитнес здоровье красота диета 
ответ полезный творог щенок 
полезный творог женский мир 
принимать творог ночь 
ответ полезный творог 
творог польза вред кто рекомендовать 
азбука ваш здоровье полезный творог 
творог польза вред полезный свойство противопо

In [12]:
import pandas as pd
train_d = pd.read_csv('train_groups.csv')
traingroups_titled = {}
for i in range(len(train_d)):
    new_doc = train_d.iloc[i]
    doc_group = new_doc['group_id']
    doc_id = new_doc['doc_id']
    target = new_doc['target']
    title = doc_to_t[doc_id]
#     title = [u.lower() for u in title if not u.lower() in list_mest]
#     title = [morph.parse(o)[0].normal_form for o in title]
    if doc_group not in traingroups_titled:
        traingroups_titled[doc_group] = []
    traingroups_titled[doc_group].append((doc_id, title, target))

In [ ]:
# %%time
# # вот этот кусок кода!!!!!!!!!!!!!!!!!!!

# import numpy as np
# import Levenshtein
# import numpy as np
# import gensim
# import nltk as nl
# from sklearn.feature_extraction.text import CountVectorizer
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.metrics.pairwise import cosine_similarity
# from gensim.models import Word2Vec
# y_train1 = []
# X_train1 = []
# groups_train1 = []
# a = 0
# b = 1000
# for new_group1 in traingroups_titled:
#     docs1 = traingroups_titled[new_group1]
#     list_data = []
#     ls = []
#     aa = 0
#     for k, (doc_id, title, target_id) in enumerate(docs1):
#         aa+=1
#         title = title.strip().split()
#         title1 = [u.lower() for u in title if not u.lower() in list_mest]
#         if len(title1)==0:
#             for e in range(10):
#                 title1.append(str(k))
#         elif len(title1)<10:
#             for e in range(10-len(title1)):
#                 title1.append(title1[0])
#         ls.append(title1[:10])
#         list_data.append(' '.join(title1))
#     print(aa)
#     print(len(ls))
#     vectorizer = TfidfVectorizer(max_features=20)
#     model = Word2Vec(ls,size = 10)
#     wordss = list(model.wv.vocab)
#     print(wordss)
#     vectorss = []
#     for word in wordss:
#         vectorss.append(model[word].tolist())
#     data11 = np.array(vectorss)
#     data11 = np.sort(data11, axis=0)
#     print(data11.shape)
#     vectorss = []
#     for word in wordss:
#         vectorss.append(model[word].tolist())
#     X6 = vectorizer.fit_transform(list_data)
#     dic = set(vectorizer.get_feature_names())
#     print(dic)
#     vectorizer = CountVectorizer(max_features=20).fit_transform(list_data)
#     vectores = vectorizer.toarray()
# #     dist = cosine_similarity(vectores, vectores)
#     dist = cosine_similarity(data11, data11)
#     print(dist.shape)
#     for k1, (doc_id1, title1, target_id1) in enumerate(docs1):
#         y_train1.append(target_id1)
#         groups_train1.append(new_group1)
#         words1 = set(title1.strip().split())
#         sum1 = 0
#         all_dist2 = []
#         all_dist4 = []
#         all_dist1 = []
#         for j in range(0, len(docs1)):
#             print(j)
#             if k1 == j:
#                 continue
#             doc_id_j, title_j, target_j = docs1[j]
#             words_j = set(title_j.strip().split())
#             y = len(words1.intersection(words_j))
#             y1 = Levenshtein.distance(title1,title_j)
#             y2 = dist[k1,j]
#             all_dist4.append(y2)
#             all_dist2.append(y1)
#             all_dist1.append(y)
#         for i in sorted(all_dist1, reverse=True)[0:15]:
#             sum1+=i
#         sum2 = 0
#         sum4 = 0
#         ss = 0
#         for i in sorted(all_dist2, reverse=False)[0:5]:
#             sum2+=i
#         for i in sorted(all_dist4, reverse=False)[0:15]:
#             sum4+=i
#         for i in sorted(all_dist2, reverse=False)[0:15]:
#             ss+=i
#         all_dist3=len(words1.intersection(dic))
#         X_train1.append(sorted(all_dist4, reverse=False)[0:15]+sorted(all_dist4, reverse=False)[15:2:25]+[all_dist3,sum4,sum1,ss,len(title1.strip().split())] + sorted(all_dist1, \
#                                 reverse=True)[0:5])
# X_train1 = np.array(X_train1)
# y_train1 = np.array(y_train1)
# groups_train1 = np.array(groups_train1)
# print (X_train1.shape, y_train1.shape, groups_train1.shape)

In [81]:
%%time
# вот этот кусок кода!!!!!!!!!!!!!!!!!!!

import numpy as np
import Levenshtein
import numpy as np
import gensim
import nltk as nl
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from gensim.models import Word2Vec
y_train1 = []
X_train1 = []
groups_train1 = []
a = 0
b = 1000
for new_group1 in traingroups_titled:
    docs1 = traingroups_titled[new_group1]
    list_data = []
    ls = []
    aa = 0
    for k, (doc_id, title, target_id) in enumerate(docs1):
        title = title.strip().split()
        title1 = [u.lower() for u in title if not u.lower() in list_mest]
        list_data.append(' '.join(title1))
    vectorizer = TfidfVectorizer(max_features=10)
    X6 = vectorizer.fit_transform(list_data)
    dic = set(vectorizer.get_feature_names())
    print(dic)
    vectorizer = CountVectorizer(max_features=20).fit_transform(list_data)
    vectores = vectorizer.toarray()
    dist = cosine_similarity(X6, X6)
    for k1, (doc_id1, title1, target_id1) in enumerate(docs1):
        y_train1.append(target_id1)
        groups_train1.append(new_group1)
        words1 = set(title1.strip().split())
        sum1 = 0
        all_dist4 = []
        for j in range(0, len(docs1)):
            if k1 == j:
                continue
            doc_id_j, title_j, target_j = docs1[j]
            words_j = set(title_j.strip().split())
            y2 = dist[k1,j]
            all_dist4.append(y2)
        sum4 = 0
        for i in sorted(all_dist4, reverse=False)[0:15]:
            sum4+=i
        all_dist3=len(words1.intersection(dic))
        X_train1.append(sorted(all_dist4, reverse=False)[0:15]+[all_dist3,sum4,len(title1.strip().split())])
X_train1 = np.array(X_train1)
y_train1 = np.array(y_train1)
groups_train1 = np.array(groups_train1)
print (X_train1.shape, y_train1.shape, groups_train1.shape)

{'цена', 'ступица', 'сравнивать', 'купить', 'подшипник', 'нива', 'ремонт', 'замена', 'товар', 'ваз'}
{'здоровье', 'народный', 'лечить', 'форум', 'вопрос', 'ларингит', 'кашель', 'лечение', 'горло', 'дневник'}
{'врач', 'онлайн', 'история', 'цикл', 'день', 'сдавать', 'библиотека', 'вопрос', 'гормон', 'ттг'}
{'экономика', 'вкладывать', 'новость', 'фонд', 'инвестиция', 'финансы', 'деньги', 'пиф', 'инвестиционный', 'стоить'}
{'андреас', 'сан', 'гта', 'онлайн', 'скачать', 'мода', 'игра', 'карта', 'код', 'сервер'}
{'правильно', 'дизайн', 'должный', 'пошив', 'тюль', 'выбирать', 'фото', 'штора', 'пол', 'длина'}
{'почка', 'народный', 'пить', 'причина', 'много', 'день', 'полезный', 'вода', 'лечение', 'дневник'}
{'симптом', 'причина', 'аллергия', 'лечить', 'форум', 'насморк', 'ребенок', 'аллергический', 'лечение', 'ринит'}
{'симптом', 'народный', 'средство', 'лечить', 'долго', 'легкое', 'лечиться', 'туберкулез', 'время', 'лечение'}
{'санандрес', 'сделать', 'гта', 'прицел', 'скачать', 'игра', 'коман

{'форум', 'устанавливать', 'гта', 'дополнение', 'скачать', 'игра', 'омск', 'симс', 'играть', 'торрент'}
{'овощ', 'свежий', 'огурец', 'возраст', 'польза', 'помидор', 'ответ', 'ребенок', 'давать', 'год'}
{'пиво', 'здоровье', 'страница', 'форум', 'женский', 'онлайн', 'женщина', 'эстроген', 'гормон', 'лечение'}
{'страница', 'бокс', 'форум', 'гта', 'прошивка', 'скачать', 'игра', 'флешка', 'торрент', 'бесплатно'}
{'форум', 'радио', 'слушать', 'онлайн', 'блог', 'сайт', 'радиолюбитель', 'эфир', 'интернет', 'радиолюбительский'}
{'дача', 'огород', 'онлайн', 'желтеть', 'помидор', 'обсуждение', 'лист', 'российский', 'сервис', 'дневник'}
{'смотреть', 'онлайн', 'кинопоиск', 'звать', 'кино', 'хороший', 'фильм', 'трейлер', 'качество', 'бесплатно'}
{'мир', 'смотреть', 'рейтинг', 'онлайн', 'самый', 'фильм', 'хороший', 'сериал', 'крутой', 'год'}
{'правильно', 'лес', 'ориентирование', 'помощь', 'компас', 'местность', 'пользоваться', 'совет', 'ориентироваться', 'карта'}
{'санпин', 'организация', 'работа', 

In [14]:
scaler = StandardScaler()
scaler.fit(X_train3)
X_train3 = scaler.transform(X_train3)

In [20]:
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn import linear_model
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score
from sklearn.linear_model import LogisticRegression

max = []
where = 0
mmm = 0
for g in range(20):
    kf = KFold(n_splits=3)
    score = np.arange(60,dtype = float).reshape(3,-1)
    a = 0
    for train,test in kf.split(X_train3):

        X_tr = X_train3[train]
        y_tr = y_train1[train]
        X_t = X_train3[test]
        y_t = y_train1[test]
#         clf = LogisticRegression(random_state=0)
        clf = MySGDClassifier(batch_generator, model_type="log_reg", max_epoch=5,
                      alpha = 0.01, C=100,batch_size = 100)
        n_estimators = 1200
        learning_rate = 0.3
        max_depth = 6
#         clf2 = XGBClassifier(n_estimators=n_estimators, learning_rate=learning_rate,
#                         max_depth=max_depth, nthread=4)
#         clf2.fit(X_tr, y_tr)
#         pr = clf2.predict(X_t)
        clf.fit(X_tr, y_tr)
        my_clf_pred = clf.predict(X_t)
        
        
        for i in range(20,40):
    #         pred = (np.array(my_clf_pred>0.01*i,dtype=int)) 
            pred = my_clf_pred
            score[a][i-20] = f1_score(pred>0.01*(i),y_t)
        a+=1
    #     break
    if g==0:
        sc = score.mean(axis = 0)
    else:
        sc += score.mean(axis = 0)
    print(g)
    print(score.mean(axis = 0).max())
#     print(g,"score",sc.max())
#     max = sc.max()
#     print(np.where(sc==max)[0][0])
#     where +=np.where(sc==max)[0][0]
# #     print(sc)
# #     print(max,'max')
#     mmm+=sc.max()
# print('mean',mmm/20)
# print('mean',where/20)
print(sc.max()/20)
print(np.where(sc==sc.max())[0][0]+20)

/home/nastya/miniconda3/envs/sphere-py37/lib/python3.7/site-packages/ipykernel_launcher.py:48: RuntimeWarning: divide by zero encountered in log
/home/nastya/miniconda3/envs/sphere-py37/lib/python3.7/site-packages/ipykernel_launcher.py:48: RuntimeWarning: invalid value encountered in multiply


0
0.7260136745286284
1
0.7297608887425567
2
0.7284633772274421
3
0.7286302119321476
4
0.7305320959790458
5
0.7275807383808406
6
0.7232908354346245
7
0.7294881881717655
8
0.7216440203910303
9
0.7303470554499288
10
0.7211456692524948
11
0.7218386440829376
12
0.6993960536903557
13
0.7249734862498459
14
0.7244942873585422
15
0.7113294743581835
16
0.7285738676467192
17
0.720076334291575
18
0.7227366228572429
19
0.7284251926764087
0.722238083734256
27


In [13]:
%%time
# вот этот кусок кода!!!!!!!!!!!!!!!!!!!

import numpy as np
import Levenshtein
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
y_train1 = []
X_train3 = []
groups_train1 = []
for new_group1 in traingroups_titled:
    docs1 = traingroups_titled[new_group1]
    list_data = []
    for k, (doc_id, title, target_id) in enumerate(docs1):
        title = title.strip().split()
        title1 = [u.lower() for u in title if not u.lower() in list_mest]
        list_data.append(' '.join(title1))
    vectorizer = TfidfVectorizer(max_features=20)
    X6 = vectorizer.fit_transform(list_data)
    dic = set(vectorizer.get_feature_names())
    print(dic)
    vectorizer = CountVectorizer(max_features=20).fit_transform(list_data)
    vectores = vectorizer.toarray()
    dist = cosine_similarity(X6, X6)
    for k1, (doc_id1, title1, target_id1) in enumerate(docs1):
        y_train1.append(target_id1)
        groups_train1.append(new_group1)
        words1 = set(title1.strip().split())
        sum1 = 0
        all_dist2 = []
        all_dist4 = []
        all_dist1 = []
        for j in range(0, len(docs1)):
            if k1 == j:
                continue
            doc_id_j, title_j, target_j = docs1[j]
            words_j = set(title_j.strip().split())
            y = len(words1.intersection(words_j))
            y1 = Levenshtein.distance(title1,title_j)
            y2 = dist[k1,j]
            all_dist4.append(y2)
            all_dist2.append(y1)
            all_dist1.append(y)
        for i in sorted(all_dist1, reverse=True)[0:15]:
            sum1+=i
        sum2 = 0
        sum4 = 0
        ss = 0
        for i in sorted(all_dist2, reverse=False)[0:5]:
            sum2+=i
        for i in sorted(all_dist4, reverse=False)[0:15]:
            sum4+=i
        for i in sorted(all_dist2, reverse=False)[0:15]:
            ss+=i
        all_dist3=len(words1.intersection(dic))
        f = 'купить' in words1
        if f==1:
            print(1)
        X_train3.append(sorted(all_dist4, reverse=False)[0:15]+sorted(all_dist4, reverse=False)[15:2:25]+[all_dist3,sum4,sum1,ss,len(title1.strip().split()),f] + sorted(all_dist1, \
                                reverse=True)[0:5])
X_train3 = np.array(X_train3)
y_train1 = np.array(y_train1)
groups_train1 = np.array(groups_train1)
print (X_train3.shape, y_train1.shape, groups_train1.shape)

{'потребительский', 'купить', 'газ', 'автомобиль', 'нива', 'ступица', 'магазин', 'замена', '2121', 'страница', 'передний', 'рука', 'подшипник', 'товар', 'ваз', 'интернет', 'ремонт', 'сравнивать', 'каталог', 'цена'}
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
{'здоровье', 'ребенок', 'домашний', 'симптом', 'российский', 'причина', 'условие', 'архив', 'дневник', 'лечить', 'сервис', 'тема', 'вопрос', 'кашель', 'народный', 'форум', 'средство', 'ларингит', 'лечение', 'горло'}
{'история', 'библиотека', 'терапия', 'гормон', 'предприятие', 'консультация', 'анализ', 'сдавать', 'ттг', 'русский', 'архив', 'день', 'страница', 'онлайн', 'вопрос', 'цикл', 'форум', 'врач', 'лечение', 'женщина'}
{'управление', 'стоить', 'инвестиционный', 'предприятие', 'бизнес', 'реферат', 'проект', 'фонд', 'россия', 'новость', 'деньги', 'экономика', 'финансы', 'выгодно', 'финансовый', 'пиф', 'форум', 'вкладывать', 'инвестиция', 'куда'}
{'скин', 'код', 'мода', 'скачать', 'гта', 'самп', 'игра', 'сервер', 'русский', 'т

{'город', 'муниципальный', 'устав', 'школа', 'учреждение', 'экстернат', 'образовательный', 'образование', 'россия', 'федеральный', 'сош', 'федерация', 'российский', 'новый', 'положение', 'деятельность', 'закон', 'москва', 'форма', 'общеобразовательный'}
{'домашний', 'сайт', 'похудение', 'рецепт', 'диета', 'избавляться', 'условие', 'быстро', 'отзыв', 'способ', 'быстрый', 'эффективный', 'народный', 'медицина', 'похудеть', 'вылечивать', 'средство', 'помощь', 'неделя', 'лечение'}
{'обзор', 'книга', 'купить', 'код', 'скачать', 'мотоцикл', 'гта', 'новость', 'характеристика', 'чистый', 'архив', 'отзыв', 'страница', 'тема', 'файл', 'сми', 'смартфон', 'видео', 'форум', 'цена'}
1
1
1
1
1
{'лист', 'распечатывать', 'таблица', 'создание', 'российский', 'информатика', 'дневник', 'процессор', 'страница', 'онлайн', 'электронный', 'тема', 'программа', 'работа', 'курсовой', 'один', 'форум', '2007', 'средство', 'бесплатно'}
1
{'рыба', 'домашний', 'блюдо', 'кулинарный', 'сушка', 'таранка', 'рецепт', 'рыба

{'омск', 'город', 'играть', 'урок', 'симс', 'сотовый', 'мода', 'устанавливать', 'скачать', 'гта', 'игра', 'русский', 'торрент', 'дополнение', 'выходить', 'онлайн', 'видео', 'форум', 'бесплатно', 'установка'}
{'возраст', 'начинать', 'польза', 'малыш', 'ребенок', 'мес', 'свежий', 'год', 'рецепт', 'огурец', 'давать', 'яйцо', 'питание', 'соленый', 'помидор', 'месяц', 'ответ', 'детский', 'мама', 'овощ'}
{'здоровье', 'гормон', 'вконтакте', 'женский', 'эстроген', 'горький', 'продукт', 'алкоголь', 'пиво', 'дневник', 'страница', 'сервис', 'витамин', 'онлайн', 'тема', 'мужчина', 'медицина', 'форум', 'лечение', 'женщина'}
{'играть', 'код', 'флешка', 'сайт', 'скачать', 'гта', 'самп', 'андроид', 'игра', 'торрент', 'бокс', 'икс', 'страница', 'чит', 'машина', 'видео', 'прошивка', '360', 'форум', 'бесплатно'}
{'радиосвязь', 'общение', 'сайт', 'любительский', 'слушать', 'радиоэфир', 'область', 'онлайн', 'радиолюбитель', 'радио', 'программа', 'приемник', 'эфир', 'радиолюбительский', 'компьютер', 'форум'

{'модем', 'кабинет', 'проверять', 'коннект', 'смарт', 'минута', 'посмотреть', 'узнавать', 'трафик', 'мтс', 'мегафон', 'личный', 'тариф', 'способ', 'оставаться', 'теле2', 'ответ', 'остаток', 'интернет', 'мобильный'}
{'полезный', 'польза', 'противопоказание', 'свойство', 'колхозница', 'торпеда', 'фрукт', 'продукт', 'арбуз', 'маленький', 'поправляться', '100', 'вред', 'жлтый', 'калория', 'дыня', 'сушеный', 'калорийность', 'ответ', 'грамм'}
{'здоровье', 'боль', 'домашний', 'симптом', 'рецепт', 'пятка', 'условие', 'фото', 'быстро', 'лечить', 'шип', 'эффективный', 'народный', 'пяточный', 'ответ', 'вылечивать', 'средство', 'шпора', 'метод', 'лечение'}
{'цена', 'противопоказание', 'артериальный', 'гипертония', 'применение', 'препарат', 'список', 'отзыв', 'блокатор', 'лекарственный', 'инструкция', 'лекарство', 'давление', 'бета', 'действие', 'болезнь', 'медицина', 'медицинский', 'показание', 'лечение'}
1
1
{'зеленеть', 'рыба', 'цвести', 'почему', 'бороться', 'появляться', 'аквариум', 'мутнеть',

In [121]:
params = product(
    (400, 600, 800, 1000, 1200), # n_estimators
    (0.01, 0.1, 0.2, 0.3, 0.4), # learning_rate
    (3, 6, 9), # max_depth
)
res_score = 0

for n_estimators, learning_rate, max_depth in list(params):
    clf = XGBClassifier(n_estimators=n_estimators, learning_rate=learning_rate,
                        max_depth=max_depth, nthread=4)
#     clf = XGBClassifier(silent=False, 
#                       scale_pos_weight=1,
#                       learning_rate=0.01,  
#                       colsample_bytree = 0.4,
#                       subsample = 0.8,
#                       objective='binary:logistic', 
#                       n_estimators=1000, 
#                       reg_alpha = 0.3,
#                       max_depth=9, 
#                       gamma=10)
    res = cross_validate(clf, X_train3, y_train1, cv=3, scoring='f1')
    score = res['test_score'].mean()
    print(score)
    print(clf)
    if score > res_score:
        res_score = score
        res_model = clf
        
res_model, res_score

0.682179675178682
XGBClassifier(base_score=None, booster=None, colsample_bylevel=None,
              colsample_bynode=None, colsample_bytree=None, gamma=None,
              gpu_id=None, importance_type='gain', interaction_constraints=None,
              learning_rate=0.01, max_delta_step=None, max_depth=3,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=400, n_jobs=None, nthread=4, num_parallel_tree=None,
              objective='binary:logistic', random_state=None, reg_alpha=None,
              reg_lambda=None, scale_pos_weight=None, subsample=None,
              tree_method=None, validate_parameters=None, verbosity=None)
0.6856052025411742
XGBClassifier(base_score=None, booster=None, colsample_bylevel=None,
              colsample_bynode=None, colsample_bytree=None, gamma=None,
              gpu_id=None, importance_type='gain', interaction_constraints=None,
              learning_rate=0.01, max_delta_step=None, max_depth=6,
    

KeyboardInterrupt: 

In [123]:
from sklearn.svm import SVC

params = product(
    (1.0,5.0,10.0,100.0), # n_estimators
    ('linear', 'poly', 'rbf', 'sigmoid'), # learning_rate
    ('scale', 'auto'), # max_depth
)
# C=1.0, kernel='rbf', degree=3, gamma='scale', coef0=0.0, shrinking=True, 
res_score = 0

for a, b, c1 in list(params):
#     clf = XGBClassifier(n_estimators=n_estimators, learning_rate=learning_rate,
#                         max_depth=max_depth, nthread=4)
    clf = SVC(gamma=c1,C = a,kernel=b)
#     clf = XGBClassifier(silent=False, 
#                       scale_pos_weight=1,
#                       learning_rate=0.01,  
#                       colsample_bytree = 0.4,
#                       subsample = 0.8,
#                       objective='binary:logistic', 
#                       n_estimators=1000, 
#                       reg_alpha = 0.3,
#                       max_depth=9, 
#                       gamma=10)
    res = cross_validate(clf, X_train3, y_train1, cv=3, scoring='f1')
    score = res['test_score'].mean()
    print(score)
    print(clf)
    if score > res_score:
        res_score = score
        res_model = clf
        
res_model, res_score

0.62816837055434
SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)
0.62816837055434
SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)
0.49688309658055
SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='poly',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)
0.4982512155470891
SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto', kernel='poly',
    max_iter=-1, probabil

0.6183532649369249
SVC(C=100.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto', kernel='sigmoid',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)


(SVC(C=10.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
     decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
     max_iter=-1, probability=False, random_state=None, shrinking=True,
     tol=0.001, verbose=False), 0.6715781676519326)

In [61]:
import numpy as np
y_train = []
X_train = []
groups_train = []
for new_group in traingroups_titled:
    docs = traingroups_titled[new_group]
    for k, (doc_id, title, target_id) in enumerate(docs):
        y_train.append(target_id)
        groups_train.append(new_group)
        all_dist = []
        words = set(title.strip().split())
        sum1 = 0
        for j in range(0, len(docs)):
            if k == j:
                continue
            doc_id_j, title_j, target_j = docs[j]
            words_j = set(title_j.strip().split())
            y = len(words.intersection(words_j))
            all_dist.append(y)
            sum1+=y
        X_train.append(sorted(all_dist, reverse=True)[0:15] +[sum1,len(title.strip().split())]   )
X_train = np.array(X_train)
y_train = np.array(y_train)
groups_train = np.array(groups_train)
print (X_train.shape, y_train.shape, groups_train.shape)

(11690, 17) (11690,) (11690,)


In [62]:
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)

In [63]:
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn import linear_model
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score
from sklearn.linear_model import LogisticRegression

max = []
where = 0
mmm = 0
for g in range(20):
    kf = KFold(n_splits=3)
    score = np.arange(60,dtype = float).reshape(3,-1)
    a = 0
    for train,test in kf.split(X_train):

        X_tr = X_train[train]
        y_tr = y_train1[train]
        X_t = X_train[test]
        y_t = y_train1[test]
#         clf = LogisticRegression(random_state=0)
        clf = MySGDClassifier(batch_generator, model_type="log_reg", max_epoch=6,
                      alpha = 0.01, C=50,batch_size = 100)
        n_estimators = 1200
        learning_rate = 0.3
        max_depth = 6
#         clf2 = XGBClassifier(n_estimators=n_estimators, learning_rate=learning_rate,
#                         max_depth=max_depth, nthread=4)
#         clf2.fit(X_tr, y_tr)
#         pr = clf2.predict(X_t)
        clf.fit(X_tr, y_tr)
        my_clf_pred = clf.predict(X_t)
        
        
        for i in range(20,40):
    #         pred = (np.array(my_clf_pred>0.01*i,dtype=int)) 
            pred = my_clf_pred
            score[a][i-20] = f1_score(pred>0.01*(i),y_t)
        a+=1
    #     break
    if g==0:
        sc = score.mean(axis = 0)
    else:
        sc += score.mean(axis = 0)
    print(g)
    print(score.mean(axis = 0).max())
#     print(g,"score",sc.max())
#     max = sc.max()
#     print(np.where(sc==max)[0][0])
#     where +=np.where(sc==max)[0][0]
# #     print(sc)
# #     print(max,'max')
#     mmm+=sc.max()
# print('mean',mmm/20)
# print('mean',where/20)
print(sc.max()/20)
print(np.where(sc==sc.max())[0][0]+20)

0
0.7147812679244424


/home/nastya/miniconda3/envs/sphere-py37/lib/python3.7/site-packages/ipykernel_launcher.py:48: RuntimeWarning: divide by zero encountered in log
/home/nastya/miniconda3/envs/sphere-py37/lib/python3.7/site-packages/ipykernel_launcher.py:48: RuntimeWarning: invalid value encountered in multiply


1
0.7171309922038865
2
0.7095076987839075
3
0.7127048537417462
4
0.7153147274069158
5
0.7145828374038103
6
0.7135820387582212
7
0.7160692404940773
8
0.7163551352065763
9
0.6439434488153455
10
0.7124890479198672
11
0.7163052204370314
12
0.7146874121183462
13
0.7093930780502679
14
0.7159533475238414
15
0.7160155082917768
16
0.7155464373044902
17
0.7177346668411569
18
0.708812399506724
19
0.7135731563590643
0.710096349364177
26


In [19]:
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.metrics import roc_auc_score
kf = KFold(n_splits=3)
score = 0
c_b = 0
a_b = 0
me_b = 0
m_b = ""
C = [10,50,100,500]
alpha = [0.01]
max_epoch = [5,6,7,8]
model_type = ["log_reg"]
for i in C:
    for j in alpha:
        for k in max_epoch:
            for z in model_type:
                a = 0
                for train,test in kf.split(X_train3):
                        X_tr = X_train3[train]
                        y_tr = y_train1[train]
                        X_t = X_train3[test]
                        y_t = y_train1[test]
                        clf = MySGDClassifier(batch_generator, model_type=z, max_epoch=k,
                                      alpha = j, C=i,batch_size = 100)
                        clf.fit(X_tr, y_tr)
                        my_clf_pred = clf.predict(X_t)
                        a = a+roc_auc_score(y_t,my_clf_pred)
                print(a/3,i,j,k,z)
                if(a/3>score):
                    score = a/3
                    c_b = i
                    a_b = j
                    me_b = k
                    m_b = z
print(score,"score")
print(c_b,"C")
print(a_b,"alpha")
print(me_b,"epoch")
print(m_b,"model_type")

/home/nastya/miniconda3/envs/sphere-py37/lib/python3.7/site-packages/ipykernel_launcher.py:48: RuntimeWarning: divide by zero encountered in log
/home/nastya/miniconda3/envs/sphere-py37/lib/python3.7/site-packages/ipykernel_launcher.py:48: RuntimeWarning: invalid value encountered in multiply
/home/nastya/miniconda3/envs/sphere-py37/lib/python3.7/site-packages/ipykernel_launcher.py:48: RuntimeWarning: divide by zero encountered in log
/home/nastya/miniconda3/envs/sphere-py37/lib/python3.7/site-packages/ipykernel_launcher.py:48: RuntimeWarning: invalid value encountered in multiply
/home/nastya/miniconda3/envs/sphere-py37/lib/python3.7/site-packages/ipykernel_launcher.py:48: RuntimeWarning: divide by zero encountered in log
/home/nastya/miniconda3/envs/sphere-py37/lib/python3.7/site-packages/ipykernel_launcher.py:48: RuntimeWarning: invalid value encountered in multiply


0.8860321093066236 10 0.01 5 log_reg


/home/nastya/miniconda3/envs/sphere-py37/lib/python3.7/site-packages/ipykernel_launcher.py:48: RuntimeWarning: divide by zero encountered in log
/home/nastya/miniconda3/envs/sphere-py37/lib/python3.7/site-packages/ipykernel_launcher.py:48: RuntimeWarning: invalid value encountered in multiply
/home/nastya/miniconda3/envs/sphere-py37/lib/python3.7/site-packages/ipykernel_launcher.py:48: RuntimeWarning: divide by zero encountered in log
/home/nastya/miniconda3/envs/sphere-py37/lib/python3.7/site-packages/ipykernel_launcher.py:48: RuntimeWarning: invalid value encountered in multiply
/home/nastya/miniconda3/envs/sphere-py37/lib/python3.7/site-packages/ipykernel_launcher.py:48: RuntimeWarning: divide by zero encountered in log
/home/nastya/miniconda3/envs/sphere-py37/lib/python3.7/site-packages/ipykernel_launcher.py:48: RuntimeWarning: invalid value encountered in multiply


0.8839447003109163 10 0.01 6 log_reg


/home/nastya/miniconda3/envs/sphere-py37/lib/python3.7/site-packages/ipykernel_launcher.py:48: RuntimeWarning: divide by zero encountered in log
/home/nastya/miniconda3/envs/sphere-py37/lib/python3.7/site-packages/ipykernel_launcher.py:48: RuntimeWarning: invalid value encountered in multiply
/home/nastya/miniconda3/envs/sphere-py37/lib/python3.7/site-packages/ipykernel_launcher.py:48: RuntimeWarning: divide by zero encountered in log
/home/nastya/miniconda3/envs/sphere-py37/lib/python3.7/site-packages/ipykernel_launcher.py:48: RuntimeWarning: invalid value encountered in multiply


...

0.886630538381239 500 0.01 6 log_reg


/home/nastya/miniconda3/envs/sphere-py37/lib/python3.7/site-packages/ipykernel_launcher.py:48: RuntimeWarning: divide by zero encountered in log
/home/nastya/miniconda3/envs/sphere-py37/lib/python3.7/site-packages/ipykernel_launcher.py:48: RuntimeWarning: invalid value encountered in multiply
/home/nastya/miniconda3/envs/sphere-py37/lib/python3.7/site-packages/ipykernel_launcher.py:48: RuntimeWarning: divide by zero encountered in log
/home/nastya/miniconda3/envs/sphere-py37/lib/python3.7/site-packages/ipykernel_launcher.py:48: RuntimeWarning: invalid value encountered in multiply
/home/nastya/miniconda3/envs/sphere-py37/lib/python3.7/site-packages/ipykernel_launcher.py:48: RuntimeWarning: divide by zero encountered in log
/home/nastya/miniconda3/envs/sphere-py37/lib/python3.7/site-packages/ipykernel_launcher.py:48: RuntimeWarning: invalid value encountered in multiply


0.8835811358355289 500 0.01 7 log_reg


/home/nastya/miniconda3/envs/sphere-py37/lib/python3.7/site-packages/ipykernel_launcher.py:48: RuntimeWarning: divide by zero encountered in log
/home/nastya/miniconda3/envs/sphere-py37/lib/python3.7/site-packages/ipykernel_launcher.py:48: RuntimeWarning: invalid value encountered in multiply
/home/nastya/miniconda3/envs/sphere-py37/lib/python3.7/site-packages/ipykernel_launcher.py:48: RuntimeWarning: divide by zero encountered in log
/home/nastya/miniconda3/envs/sphere-py37/lib/python3.7/site-packages/ipykernel_launcher.py:48: RuntimeWarning: invalid value encountered in multiply
/home/nastya/miniconda3/envs/sphere-py37/lib/python3.7/site-packages/ipykernel_launcher.py:48: RuntimeWarning: divide by zero encountered in log
/home/nastya/miniconda3/envs/sphere-py37/lib/python3.7/site-packages/ipykernel_launcher.py:48: RuntimeWarning: invalid value encountered in multiply


0.8805620811619056 500 0.01 8 log_reg
0.8873267326493274 score
100 C
0.01 alpha
5 epoch
log_reg model_type


In [52]:
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
max = []
where = 0
mmm = 0
for g in range(20):
    kf = KFold(n_splits=3)
    score = np.arange(60,dtype = float).reshape(3,-1)
    a = 0
    for train,test in kf.split(X_train):

        X_tr = X_train[train]
        y_tr = y_train1[train]
        X_t = X_train[test]
        y_t = y_train1[test]
        X_tr1 = X_train1[train]
        X_t1 = X_train1[test]
#         clf = LogisticRegression(random_state=0)
        clf = MySGDClassifier(batch_generator, model_type="log_reg", max_epoch=6,
                      alpha = 0.01, C=50,batch_size = 100)
        clf.fit(X_tr, y_tr)
        my_clf_pred = clf.predict(X_t)
        res1 = (my_clf_pred>0.01*(26)).reshape(-1,1)
        
        
        
        
        n_estimators = 1200
        learning_rate = 0.001
        max_depth = 6
        clf2 = XGBClassifier(n_estimators=n_estimators, learning_rate=learning_rate,
                        max_depth=max_depth, nthread=4)
        clf2.fit(X_tr, y_tr)
        pr = clf2.predict(X_t).reshape(-1,1)
        clf1 = MySGDClassifier(batch_generator, model_type="log_reg", max_epoch=6,
                      alpha = 0.01, C=50,batch_size = 100)
        clf1.fit(X_tr1, y_tr)
        my_clf_pred1 = clf1.predict(X_t1)
        res2 = (my_clf_pred1>0.01*(28)).reshape(-1,1)
        clf3 = XGBClassifier(n_estimators=n_estimators, learning_rate=learning_rate,
                        max_depth=max_depth, nthread=4)
        clf3.fit(X_tr1, y_tr)
        pr1 = clf3.predict(X_t1).reshape(-1,1)
        
        X_res = np.concatenate((res1,pr,res2,pr1), axis=1)
        y_res =  (np.mean(X_res, axis = 1))>0.5
        print(X_res.shape)
        print(y_res.shape)
        print(y_t.shape)
        for i in range(20,40):
    #         pred = (np.array(my_clf_pred>0.01*i,dtype=int)) 
            pred = my_clf_pred
            score[a][i-20] = f1_score(y_res,y_t)
        a+=1
    #     break
    if g==0:
        sc = score.mean(axis = 0)
    else:
        sc += score.mean(axis = 0)
    print(g)
    print(score.mean(axis = 0).max())
#     print(g,"score",sc.max())
#     max = sc.max()
#     print(np.where(sc==max)[0][0])
#     where +=np.where(sc==max)[0][0]
# #     print(sc)
# #     print(max,'max')
#     mmm+=sc.max()
# print('mean',mmm/20)
# print('mean',where/20)
print(sc.max()/20)
print(np.where(sc==sc.max())[0][0]+20)

/home/nastya/miniconda3/envs/sphere-py37/lib/python3.7/site-packages/ipykernel_launcher.py:48: RuntimeWarning: divide by zero encountered in log
/home/nastya/miniconda3/envs/sphere-py37/lib/python3.7/site-packages/ipykernel_launcher.py:48: RuntimeWarning: invalid value encountered in multiply


(3897, 4)
(3897,)
(3897,)


KeyboardInterrupt: 

In [64]:
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
max = []
where = 0
mmm = 0
www = np.zeros(4)
for g in range(5):
    kf = KFold(n_splits=3)
    score = np.arange(60,dtype = float).reshape(3,-1)
    a = 0
    for train,test in kf.split(X_train):

        X_tr = X_train[train]
        y_tr = y_train1[train]
        X_t = X_train[test]
        y_t = y_train1[test]
        X_tr1 = X_train1[train]
        X_t1 = X_train1[test]
#         clf = LogisticRegression(random_state=0)
        clf = MySGDClassifier(batch_generator, model_type="log_reg", max_epoch=6,
                      alpha = 0.01, C=50,batch_size = 100)
        clf.fit(X_tr, y_tr)
        my_clf_pred = clf.predict(X_t)
        res1 = (my_clf_pred>0.01*(26)).reshape(-1,1)
        
        
        
        
        n_estimators = 1200
        learning_rate = 0.001
        max_depth = 6
        clf2 = XGBClassifier(n_estimators=n_estimators, learning_rate=learning_rate,
                        max_depth=max_depth, nthread=4)
        clf2.fit(X_tr, y_tr)
        pr = clf2.predict(X_t).reshape(-1,1)
        clf1 = MySGDClassifier(batch_generator, model_type="log_reg", max_epoch=6,
                      alpha = 0.01, C=50,batch_size = 100)
        clf1.fit(X_tr1, y_tr)
        my_clf_pred1 = clf1.predict(X_t1)
        res2 = (my_clf_pred1>0.01*(29)).reshape(-1,1)
        clf3 = XGBClassifier(n_estimators=n_estimators, learning_rate=learning_rate,
                        max_depth=max_depth, nthread=4)
        clf3.fit(X_tr1, y_tr)
        pr1 = clf3.predict(X_t1).reshape(-1,1)
        
        X_res = np.concatenate((res1,pr,res2,pr1), axis=1)
#         y_res =  (np.mean(X_res, axis = 1))>0.5
        print(X_res.shape)
        print(y_res.shape)
        print(y_t.shape)
        clf5 = MySGDClassifier(batch_generator, model_type="lin_reg", max_epoch=6,
                      alpha = 0.01, C=50,batch_size = 100)
        reg = LinearRegression().fit(X_res, y_t)
        www+=reg.coef_
        print(www/(g*3+3))
#     print(g,"score",sc.max())
#     max = sc.max()
#     print(np.where(sc==max)[0][0])
#     where +=np.where(sc==max)[0][0]
# #     print(sc)
# #     print(max,'max')
#     mmm+=sc.max()
# print('mean',mmm/20)
# print('mean',where/20)
print(www/10)

(3897, 4)
(3897, 1)
(3897,)
[0.07167395 0.03481372 0.06852429 0.05661568]


/home/nastya/miniconda3/envs/sphere-py37/lib/python3.7/site-packages/ipykernel_launcher.py:48: RuntimeWarning: divide by zero encountered in log
/home/nastya/miniconda3/envs/sphere-py37/lib/python3.7/site-packages/ipykernel_launcher.py:48: RuntimeWarning: invalid value encountered in multiply


(3897, 4)
(3897, 1)
(3897,)
[0.14806138 0.10264635 0.10317016 0.12068825]


/home/nastya/miniconda3/envs/sphere-py37/lib/python3.7/site-packages/ipykernel_launcher.py:48: RuntimeWarning: divide by zero encountered in log
/home/nastya/miniconda3/envs/sphere-py37/lib/python3.7/site-packages/ipykernel_launcher.py:48: RuntimeWarning: invalid value encountered in multiply


(3896, 4)
(3897, 1)
(3896,)
[0.1749663  0.17144779 0.16566921 0.21521665]


...

(3896, 4)
(3897, 1)
(3896,)
[0.19565932 0.16159799 0.16386877 0.20445488]
(3897, 4)
(3897, 1)
(3897,)
[0.16777848 0.13660315 0.1463222  0.17652838]


/home/nastya/miniconda3/envs/sphere-py37/lib/python3.7/site-packages/ipykernel_launcher.py:48: RuntimeWarning: divide by zero encountered in log
/home/nastya/miniconda3/envs/sphere-py37/lib/python3.7/site-packages/ipykernel_launcher.py:48: RuntimeWarning: invalid value encountered in multiply


(3897, 4)
(3897, 1)
(3897,)
[0.1851152  0.15009925 0.15124494 0.18943829]


/home/nastya/miniconda3/envs/sphere-py37/lib/python3.7/site-packages/ipykernel_launcher.py:48: RuntimeWarning: divide by zero encountered in log
/home/nastya/miniconda3/envs/sphere-py37/lib/python3.7/site-packages/ipykernel_launcher.py:48: RuntimeWarning: invalid value encountered in multiply


(3896, 4)
(3897, 1)
(3896,)
[0.19721109 0.16177852 0.15989065 0.20706869]
[0.29581664 0.24266778 0.23983597 0.31060303]


In [38]:
www/30

array([0.22154872, 0.14664111, 0.14601091, 0.20563443])

In [65]:
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
max = []
where = 0
mmm = 0
for g in range(20):
    kf = KFold(n_splits=3)
    score = np.arange(300,dtype = float).reshape(3,-1)
    a = 0
    for train,test in kf.split(X_train):

        X_tr = X_train[train]
        y_tr = y_train1[train]
        X_t = X_train[test]
        y_t = y_train1[test]
        X_tr1 = X_train1[train]
        X_t1 = X_train1[test]
#         clf = LogisticRegression(random_state=0)
        clf = MySGDClassifier(batch_generator, model_type="log_reg", max_epoch=6,
                      alpha = 0.01, C=50,batch_size = 100)
        clf.fit(X_tr, y_tr)
        my_clf_pred = clf.predict(X_t)
        res1 = (my_clf_pred>0.01*(26)).reshape(-1,1)
        
        
        
        
        n_estimators = 1200
        learning_rate = 0.001
        max_depth = 6
        clf2 = XGBClassifier(n_estimators=n_estimators, learning_rate=learning_rate,
                        max_depth=max_depth, nthread=4)
        clf2.fit(X_tr, y_tr)
        pr = clf2.predict(X_t).reshape(-1,1)
        clf1 = MySGDClassifier(batch_generator, model_type="log_reg", max_epoch=6,
                      alpha = 0.01, C=50,batch_size = 100)
        clf1.fit(X_tr1, y_tr)
        my_clf_pred1 = clf1.predict(X_t1)
        res2 = (my_clf_pred1>0.01*(29)).reshape(-1,1)
        clf3 = XGBClassifier(n_estimators=n_estimators, learning_rate=learning_rate,
                        max_depth=max_depth, nthread=4)
        clf3.fit(X_tr1, y_tr)
        pr1 = clf3.predict(X_t1).reshape(-1,1)
        
        X_res = np.concatenate((res1,pr,res2,pr1), axis=1)
        y_res =  res1*0.29581664+pr* 0.24266778+res2* 0.23983597+pr1* 0.20563443
#         print(X_res.shape)
#         print(y_res.shape)
#         print(y_t.shape)
        for i in range(0,100):
    #         pred = (np.array(my_clf_pred>0.01*i,dtype=int)) 
            pred = my_clf_pred
            score[a][i] = f1_score(y_res>0.01,y_t)
        a+=1
    #     break
    if g==0:
        sc = score.mean(axis = 0)
    else:
        sc += score.mean(axis = 0)
    print(g)
    print(score.mean(axis = 0).max())
#     print(g,"score",sc.max())
#     max = sc.max()
#     print(np.where(sc==max)[0][0])
#     where +=np.where(sc==max)[0][0]
# #     print(sc)
# #     print(max,'max')
#     mmm+=sc.max()
# print('mean',mmm/20)
# print('mean',where/20)
print(sc.max()/20)
print(np.where(sc==sc.max())[0][0])

/home/nastya/miniconda3/envs/sphere-py37/lib/python3.7/site-packages/ipykernel_launcher.py:48: RuntimeWarning: divide by zero encountered in log
/home/nastya/miniconda3/envs/sphere-py37/lib/python3.7/site-packages/ipykernel_launcher.py:48: RuntimeWarning: invalid value encountered in multiply


0
0.7188433406582219
1
0.7059350432772873
2
0.7185650851114782
3
0.7166568719300007
4
0.7217715102771858
5
0.7238488313578063
6
0.7177604271392327
7
0.7188453819155849


KeyboardInterrupt: 

In [16]:
test_data1 = pd.read_csv('test_groups.csv')
test_groups_titled1 = {}
for i in range(len(test_data1)):
    new_doc1 = test_data1.iloc[i]
    doc_group1 = new_doc1['group_id']
    doc_id1 = new_doc1['doc_id']
    title1 = doc_to_t[doc_id1]
    
#     title1 = [u.lower() for u in title1 if not u.lower() in list_mest]
#     title1 = [morph.parse(o)[0].normal_form for o in title1]
    if doc_group1 not in test_groups_titled1:
        test_groups_titled1[doc_group1] = []
    test_groups_titled1[doc_group1].append((doc_id1, title1))

In [17]:
%%time
X_test1 = []
groups_test1 = []
p1 = 0
for new_group in test_groups_titled1:
    p1+=1
    print(p1)
    docs1 = test_groups_titled1[new_group]
    list_data = []
    for k, (doc_id, title) in enumerate(docs1):
        title = title.strip().split()
        title1 = [u.lower() for u in title if not u.lower() in list_mest]
        list_data.append(' '.join(title1))
    vectorizer = TfidfVectorizer(max_features=20)
    X6 = vectorizer.fit_transform(list_data)
    dic = set(vectorizer.get_feature_names())
    print(dic)
    vectorizer = CountVectorizer(max_features=20).fit_transform(list_data)
    vectores = vectorizer.toarray()
    dist = cosine_similarity(X6, X6)
    for k, (doc_id, title) in enumerate(docs1):
        groups_test1.append(new_group)
        all_dist4 = []
        all_dist2 = []
        words = set(title.strip().split())
        sum1 = 0
#         dist = np.zeros((len(docs1), len(docs1)))
        all_dist1 = []
        n = 0
        for j in range(0, len(docs1)):
            if k == j:
                continue
            n+=1
            doc_id_j, title_j= docs1[j]
            words_j = set(title_j.strip().split())
            y = len(words.intersection(words_j))
            y1 = Levenshtein.distance(title,title_j)
            all_dist2.append(y1)
            y2 = dist[k,j]
#             print(y2)
            all_dist4.append(y2)
            all_dist1.append(y)
#             sum1+=y
        for i in sorted(all_dist1, reverse=True)[0:15]:
            sum1+=i
        sum2 = 0
        for i in sorted(all_dist2, reverse=False)[0:5]:
            sum2+=i
        sum4 = 0
        for i in sorted(all_dist4, reverse=False)[0:15]:
            sum4+=i
        ss = 0
        for i in sorted(all_dist2, reverse=False)[0:15]:
            ss+=i
        all_dist3=len(words.intersection(dic))
        f = 'купить' in words
        X_test1.append(sorted(all_dist4, reverse=False)[0:15] +sorted(all_dist4, reverse=False)[15:2:25]+[all_dist3, sum4,sum1,ss,len(title.strip().split()),f] +sorted(all_dist1, reverse=True)[0:5]  )
#         print(sum2)
X_test1 = np.array(X_test1)
groups_test1 = np.array(groups_test1)
print (X_test1.shape, groups_test1.shape)

1
{'плагин', 'админка', 'сайт', 'скачать', 'гта', 'самп', 'создавать', 'статья', 'игра', 'сервер', 'админ', 'прописывать', 'давать', 'ксс', 'страница', 'видео', 'форум', 'помощь', 'интернет', 'блог'}
2
{'контакт', 'адрес', 'добираться', 'метро', 'россия', 'торговый', 'телефон', 'магазин', 'центр', 'отзыв', 'сити', 'группа', 'вакансия', 'работа', 'строгино', 'мякинино', 'москва', 'компания', 'международный', 'крокус'}
3
{'точный', 'балашовский', 'год', 'россия', 'новость', '2017', 'балашов', 'русский', 'погода', 'день', 'дневник', 'область', 'район', 'прогноз', 'тема', '2016', 'почасовой', 'мир', 'самый', 'саратовский'}
4
{'книга', 'магия', 'ядовитый', 'борщевик', 'новость', 'обсуждение', 'трава', 'ожог', 'растение', 'российский', 'дневник', 'лечить', 'сервис', 'страница', 'онлайн', 'тема', 'форум', 'бесплатно', 'мир', 'лечение'}
5
{'протертый', 'вконтакте', 'домашний', 'сахар', 'кулинарный', 'рецепт', 'обсуждение', 'магазин', 'российский', 'заготовка', 'смородина', 'фото', 'дневник', '

40
{'глаз', 'солнцезащитный', 'оптика', 'выбирать', 'контактный', 'зрение', 'рыбалка', 'телефон', 'бифокальный', 'салон', 'отзыв', 'дневник', 'линза', 'очки', 'москва', 'интернет', 'медицинский', 'подбор', 'цена', 'лечение'}
41
{'комаровский', 'ребенок', 'дибазол', 'скачать', 'применение', 'помогать', 'педиатрия', 'отзыв', 'страница', 'инструкция', 'лекарство', 'совет', 'миопатия', 'инфекционный', 'форум', 'средство', 'врач', 'бесплатно', 'показание', 'лечение'}
42
{'2010', 'колледж', 'сайт', 'учреждение', 'год', 'образование', 'образовательный', 'новость', 'комиссия', 'бюджетный', 'область', 'страница', 'деятельность', 'педагогический', 'вуз', 'средний', 'государственный', 'университет', 'научно', 'профессиональный'}
43
{'купить', 'модем', 'роутер', 'настройка', 'сменить', 'посмотреть', 'узнавать', 'телефон', 'дешево', 'китайский', '300', 'поставлять', 'забывать', 'фай', 'компьютер', 'настраивать', 'ответ', 'поменять', 'сеть', 'пароль'}
44
{'библиотека', 'налогообложение', 'предприяти

80
{'история', 'регистрация', 'юрист', 'зарегистрировать', 'авторский', 'глаголать', 'уголовный', 'право', 'дело', 'архив', 'дневник', 'страница', 'онлайн', 'тема', 'гражданин', 'вопрос', 'международный', 'форум', 'бесплатно', 'мир'}
81
{'обзор', 'предмет', 'код', 'симс', 'мода', 'горячий', 'скачать', 'навык', 'деньги', 'игра', 'увеличивать', 'секрет', 'дополнение', 'размер', 'грудь', 'чит', 'форум', 'чита', 'бесплатно', 'блог'}
82
{'рыба', 'сильно', 'домашний', 'блюдо', 'еда', 'кулинарный', 'спасать', 'рецепт', 'пища', 'пересолить', 'соль', 'гречка', 'исправлять', 'каша', 'фото', 'делать', 'совет', 'подсказка', 'ответ', 'суп'}
83
{'февраль', 'поздравление', 'годовщина', 'отношение', 'стих', 'любимый', 'студент', 'день', 'поздравлять', 'муж', 'оригинально', 'идея', 'мужчина', 'ответ', 'форум', 'рождение', 'оригинальный', 'парень', 'днм', 'друг'}
84
{'2013', 'сериал', 'роль', 'класс', 'жизнь', 'актер', 'качество', 'биография', 'хороший', 'новость', 'смотреть', 'страница', 'мелодрама', '

119
{'связь', 'клиент', 'дозваниваться', 'сотовый', 'подмосковье', 'частный', 'позвонить', 'номер', 'телефон', 'мтс', 'тариф', 'область', 'телевидение', 'услуга', 'москва', 'ответ', 'интернет', 'бесплатно', 'оператор', 'мобильный'}
120
{'фасон', 'синий', 'мода', 'стильный', 'женский', 'воротник', 'белый', 'выбирать', 'разговор', 'модный', 'территория', 'фото', 'жакет', 'носить', 'совет', '2014', 'модель', 'стиль', 'женщина', 'пиджак'}
121
{'новинка', 'качество', 'скачать', 'год', 'хороший', 'новость', '2017', 'русский', 'торрент', 'смотреть', 'новый', 'ужас', 'боевик', 'онлайн', 'серия', '720', 'кино', '2016', 'фильм', 'бесплатно'}
122
{'млн', 'муниципальный', 'бизнес', 'кроссовер', 'автомобиль', 'год', 'россия', 'хороший', 'новость', '2017', 'клуб', 'казань', 'архив', 'экономика', 'финансы', 'район', 'авто', 'рубль', 'политика', 'форум'}
123
{'почему', 'скачивать', 'перекачивать', 'сайт', 'скачать', 'андроид', 'чоп', 'новость', 'игра', 'клуб', 'обновление', 'телефон', 'архив', 'другой

159
{'1068', 'драйвер', 'служба', 'сайт', 'скачать', 'андроид', 'игра', 'обсуждение', 'архив', 'дочерний', 'запускать', 'страница', 'удаваться', 'вопрос', 'компьютер', 'форум', 'русски', 'система', 'ошибка', 'вирус'}
160
{'беременность', 'ребенок', 'домашний', 'россия', 'выкидыш', 'матка', 'препарат', 'российский', 'беременный', 'месячный', 'условие', 'дневник', 'онлайн', 'аборт', 'вопрос', 'народный', 'форум', 'вызывать', 'средство', 'лечение'}
161
{'сегодня', 'единый', 'добираться', 'крым', 'поезд', 'дешевый', 'новость', '2017', 'дешево', 'маршрут', 'симферополь', 'украина', 'дорога', 'отдых', 'авиабилет', 'москва', 'билет', 'расписание', 'цена', 'доезжать'}
162
{'здоровье', 'долго', 'ребенок', 'симптом', 'реферат', 'язва', 'российский', 'причина', 'заживать', 'дневник', 'сервис', 'онлайн', 'заболевание', 'болезнь', 'народный', 'медицина', 'текст', 'форум', 'рана', 'лечение'}
163
{'проблема', 'переустановить', 'звуковой', 'драйвер', 'устанавливать', 'аудио', 'скачать', 'статья', 'ноу

In [18]:
scaler = StandardScaler()
scaler.fit(X_test1)
X_test1 = scaler.transform(X_test1)

In [27]:
# score на kaggle 0,705
# модель с косинусной схожестью(ансамбль моделей и word2vec пока не делала), использовала my stem вместо pymorphy
# добавила фичу наличие слова 'купить'
clf1 = MySGDClassifier(batch_generator, model_type="log_reg", max_epoch=5,
                  alpha = 0.01, C=100,batch_size = 100)
clf1.fit(X_train3, y_train1)
my_clf_pred1 = clf1.predict(X_test1)
# clf = MySGDClassifier(batch_generator, model_type="log_reg", max_epoch=10,
#                   alpha = 0.01, C=1,batch_size = 100)
# clf.fit(X_train, y_train)
# my_clf_pred = clf.predict(X_test)
pred = my_clf_pred1 
print(np.array(pred>0.29,dtype = int).sum())
test_data1["target"] = pd.Series(np.array(pred>0.29,dtype = int))
with open("pred_1227.csv", "w") as f:
    f.write(test_data1.to_csv(columns=("pair_id", "target"), index=False))

/home/nastya/miniconda3/envs/sphere-py37/lib/python3.7/site-packages/ipykernel_launcher.py:48: RuntimeWarning: divide by zero encountered in log
/home/nastya/miniconda3/envs/sphere-py37/lib/python3.7/site-packages/ipykernel_launcher.py:48: RuntimeWarning: invalid value encountered in multiply


4908


In [28]:
y_train1.sum()/y_train1.shape[0]

0.28751069289991443

In [29]:
(pred>0.29).sum()/pred.shape[0]

0.29518253443194803